* This sript is based on the script on 20160603
* The purpose of this script today is to study the combination of different models

In [131]:
import numpy as np
import matplotlib.pyplot as plt

import graphlab
import datetime
import math

from sklearn.ensemble import GradientBoostingRegressor
from sklearn import ensemble
from sklearn import datasets
from sklearn.utils import shuffle
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

%matplotlib inline

np.random.seed(1)

### Define functions

In [132]:
# datetime.date format: year, month, day
def date_to_day(date):
    date_list = date.strip().split('-')
    return datetime.date(int(date_list[0]),int(date_list[1]),int(date_list[2])).weekday() + 1

In [133]:
# define MAPE evaluation function
def mape(result, result_predicted):
    count = 0
    sum = 0
    for (item_1, item_2) in zip(result, result_predicted):
        if item_1 > 0:
            count = count + 1
            sum = sum + math.fabs((item_1 - item_2)/item_1)
    return (sum, count, sum/count)

In [134]:
# define post_prediction_modification to predicted data
# set all the predicted gap that is less than 1 to 1
def prediction_modification(prediction):
    vect = prediction
    for i in range(len(vect)):
        if vect[i] < 1.0:
            vect[i] = 1.0
    return vect

### Load data

In [135]:
# SFrame.read_csv function to read csv files
data_training = graphlab.SFrame.read_csv("data-all_training.csv", \
                                        column_type_hints=[str,str,long,long,long,long,long,float,long,long,long,long,\
                                                           float,float,float,float,float,str,str,str,str,long,long,\
                                                           long,long,float,long,long,long,long,float,long,long,long,long,\
                                                           long,long,long,long,long,long,long,long,long,long,long,long])
data_test_set_1 = graphlab.SFrame.read_csv("data-all_test_set_1.csv", \
                                          column_type_hints=[str,str,long,long,long,long,long,float,long,long,long,long,\
                                                           float,float,float,float,float,str,str,str,str,long,long,\
                                                           long,long,float,long,long,long,long,float,long,long,long,long,\
                                                           long,long,long,long,long,long,long,long,long,long,long,long])

Finished parsing file D:\Di-Tech_Challenge\Machine_Learning\Model_20160608\data-all_training.csv

Parsing completed. Parsed 199584 lines in 1.06006 secs.

Finished parsing file D:\Di-Tech_Challenge\Machine_Learning\Model_20160608\data-all_test_set_1.csv

Parsing completed. Parsed 47520 lines in 0.243014 secs.

In [136]:
data_training['day_of_week'] = data_training['date'].apply(lambda x: str(date_to_day(x)))
data_test_set_1['day_of_week'] = data_test_set_1['date'].apply(lambda x: str(date_to_day(x)))

data_training['gap_delta'] = data_training['gap_t(j)'] - data_training['gap_t(j-1)']
data_training['gap_delta_1'] = data_training['gap_t(j-1)'] - data_training['gap_t(j-2)']
data_training['gap_delta_2'] = data_training['gap_t(j-2)'] - data_training['gap_t(j-3)']
data_training['gap_delta_3'] = data_training['gap_t(j-1)'] - data_training['gap_t(j-3)']

data_test_set_1['gap_delta'] = data_test_set_1['gap_t(j)'] - data_test_set_1['gap_t(j-1)']
data_test_set_1['gap_delta_1'] = data_test_set_1['gap_t(j-1)'] - data_test_set_1['gap_t(j-2)']
data_test_set_1['gap_delta_2'] = data_test_set_1['gap_t(j-2)'] - data_test_set_1['gap_t(j-3)']
data_test_set_1['gap_delta_3'] = data_test_set_1['gap_t(j-1)'] - data_test_set_1['gap_t(j-3)']

# delete data at time_slot_id 1, 2 and 3
data_training = data_training[(data_training['time_slot_id'] > 3)]

In [137]:
data_training_training, data_training_validation = data_training.random_split(.9, seed=8)

In [138]:
features = ['start_district_id', 'time_slot_id', 'gap_t(j-1)', 'gap_t(j-2)', 'gap_t(j-3)', 'gap_averaged',
            'gap_delta_1','gap_delta_2','gap_delta_3',
            'order_t(j-1)', 'order_t(j-2)', 'order_t(j-3)', 'order_averaged', 'price_avg_t(j-1)', 'price_avg_t(j-2)',
            'price_avg_t(j-3)', 'weather_t(j)', 'weather_t(j-1)', 'weather_t(j-2)', 'weather_t(j-3)', 
            'pm_t(j)','pm_t(j-1)','pm_t(j-2)','pm_t(j-3)','pm_avg', 'tj_1_j', 'tj_2_j','tj_3_j','tj_4_j',
            'tj_1_j_1', 'tj_2_j_1','tj_3_j_1','tj_4_j_1', 'tj_1_j_2', 'tj_2_j_2','tj_3_j_2','tj_4_j_2',
            'tj_1_j_3', 'tj_2_j_3','tj_3_j_3','tj_4_j_3','day_of_week']

In [139]:
feature_numpy_training = data_training_training[features].to_numpy().astype(float)

In [140]:
len(feature_numpy_training)

176005

In [141]:
target_numpy_training = data_training_training['gap_delta'].to_numpy().astype(float)

In [142]:
feature_numpy_validation = data_training_validation[features].to_numpy().astype(float)

In [143]:
target_numpy_validation = data_training_validation['gap_delta'].to_numpy().astype(float)

### Explore hyperparameters

In [144]:
'''
# define parameters to search

n_estimators = [5,10,20,30,40,50,60,70,80,90,100,150,200,300,400,500]
max_depth = [2,3,4,5,6,7,8]
min_samples_split = [1,2,3,4]

n_estimators = [5,10]
max_depth = [2,3]
min_samples_split = [1,2]
'''

'\n# define parameters to search\n\nn_estimators = [5,10,20,30,40,50,60,70,80,90,100,150,200,300,400,500]\nmax_depth = [2,3,4,5,6,7,8]\nmin_samples_split = [1,2,3,4]\n\nn_estimators = [5,10]\nmax_depth = [2,3]\nmin_samples_split = [1,2]\n'

In [145]:
'''
output_file = open("hyperparamters_test_result",'w')
output_file.write("n_estimators,max_depth,min_samples_split,mae,rmse\n")
output_file.close()

for i in range(len(n_estimators)):
    for j in range(len(max_depth)):
        for k in range(len(min_samples_split)):
            print i, j, k
            params = {'n_estimators':n_estimators[i], 'max_depth': max_depth[j], 'min_samples_split': min_samples_split[k], 
                      'verbose': False, 'learning_rate': 0.01, 'loss': 'quantile', 'alpha': 0.5}
            model = ensemble.GradientBoostingRegressor(**params)
            model.fit(feature_numpy_training, target_numpy_training)
            validation_prediction = model.predict(feature_numpy_validation).tolist()
            validation_target = target_numpy_validation.tolist()
            (sum, count, mae) = mape(validation_target,validation_prediction)
            rmse = np.sqrt(mean_squared_error(validation_target,validation_prediction))
            output_file = open("hyperparamters_test_result",'a')
            output_file.write(str(n_estimators[i]) + ',' + str(max_depth[j]) + ',' + str(min_samples_split[k]) + \
                              ',' + str(mae) + ',' + str(rmse) + '\n')
            output_file.close()
            
 '''           

'\noutput_file = open("hyperparamters_test_result",\'w\')\noutput_file.write("n_estimators,max_depth,min_samples_split,mae,rmse\n")\noutput_file.close()\n\nfor i in range(len(n_estimators)):\n    for j in range(len(max_depth)):\n        for k in range(len(min_samples_split)):\n            print i, j, k\n            params = {\'n_estimators\':n_estimators[i], \'max_depth\': max_depth[j], \'min_samples_split\': min_samples_split[k], \n                      \'verbose\': False, \'learning_rate\': 0.01, \'loss\': \'quantile\', \'alpha\': 0.5}\n            model = ensemble.GradientBoostingRegressor(**params)\n            model.fit(feature_numpy_training, target_numpy_training)\n            validation_prediction = model.predict(feature_numpy_validation).tolist()\n            validation_target = target_numpy_validation.tolist()\n            (sum, count, mae) = mape(validation_target,validation_prediction)\n            rmse = np.sqrt(mean_squared_error(validation_target,validation_prediction))\

### Train the model

In [146]:
# Least Absolute Deviation (LAD) regression

# Fit regression model
params = {'n_estimators':100, 'max_depth': 4, 'min_samples_split': 1, 'verbose': True,
          'learning_rate': 0.01, 'loss': 'quantile', 'alpha': 0.5}
model_lad = ensemble.GradientBoostingRegressor(**params)

model_lad.fit(feature_numpy_training, target_numpy_training)


      Iter       Train Loss   Remaining Time 
         1          -0.0016            2.28m
         2          -0.0008            2.20m
         3           0.0005            2.38m
         4           0.0017            2.45m
         5           0.0032            2.35m
         6           0.0045            2.29m
         7           0.0057            2.24m
         8           0.0065            2.18m
         9           0.0078            2.14m
        10           0.0091            2.10m
        20           0.0210            1.79m
        30           0.0323            1.53m
        40           0.0425            1.30m
        50           0.0514            1.07m
        60           0.0595           51.72s
        70           0.0668           39.02s
        80           0.0717           26.03s
        90           0.0754           13.03s
       100           0.0785            0.00s


GradientBoostingRegressor(alpha=0.5, init=None, learning_rate=0.01,
             loss='quantile', max_depth=4, max_features=None,
             max_leaf_nodes=None, min_samples_leaf=1, min_samples_split=1,
             min_weight_fraction_leaf=0.0, n_estimators=100,
             presort='auto', random_state=None, subsample=1.0,
             verbose=True, warm_start=False)

In [147]:
# Least Squares (LS) regression

# Fit regression model
params = {'n_estimators':1, 'max_depth': 4, 'min_samples_split': 1, 'verbose': True,
          'learning_rate': 0.01, 'loss': 'ls'}
model_ls = ensemble.GradientBoostingRegressor(**params)

model_ls.fit(feature_numpy_training, target_numpy_training)


      Iter       Train Loss   Remaining Time 
         1         253.7518            0.00s


GradientBoostingRegressor(alpha=0.9, init=None, learning_rate=0.01, loss='ls',
             max_depth=4, max_features=None, max_leaf_nodes=None,
             min_samples_leaf=1, min_samples_split=1,
             min_weight_fraction_leaf=0.0, n_estimators=1, presort='auto',
             random_state=None, subsample=1.0, verbose=True,
             warm_start=False)

In [148]:
# Graphlab model
model_gl = graphlab.boosted_trees_regression.create(data_training_training, features=features, target='gap_delta', 
                                                    max_iterations = 100,
                                                    max_depth = 9, random_seed = 1)

PROGRESS: Creating a validation set from 5 percent of training data. This may take a while.
          You can set ``validation_set=None`` to disable validation tracking.



Boosted trees regression:

--------------------------------------------------------

Number of examples          : 167179

Number of features          : 42

...

| 85        | 29.595692    | 163.959885         | 6.080530      | 699.243652           | 14.685535       |

| 90        | 31.245787    | 164.007660         | 6.033166      | 699.198242           | 14.677829       |

| 95        | 33.222900    | 163.905319         | 5.977470      | 699.095886           | 14.676167       |

| 100       | 35.047004    | 163.908539         | 5.934249      | 699.099121           | 14.692945       |

+-----------+--------------+--------------------+---------------+----------------------+-----------------+

In [149]:
# Least Absolute Deviation (LAD) regression
# Guotu features


features_guotu = ['start_district_id', 'time_slot_id', 'gap_t(j-1)', 'gap_t(j-2)', 'gap_t(j-3)', 
                  'order_t(j-1)', 'order_t(j-2)', 'order_t(j-3)', 
                  'weather_t(j)', 'weather_t(j-1)', 'weather_t(j-2)', 'weather_t(j-3)', 
                  'temperature_t(j)', 'temperature_t(j-1)','temperature_t(j-2)','temperature_t(j-3)', 
                  'pm_t(j)','pm_t(j-1)','pm_t(j-2)','pm_t(j-3)', 'tj_1_j', 'tj_2_j','tj_3_j','tj_4_j',
                  'tj_1_j_1', 'tj_2_j_1','tj_3_j_1','tj_4_j_1', 'tj_1_j_2', 'tj_2_j_2','tj_3_j_2','tj_4_j_2',
                  'tj_1_j_3', 'tj_2_j_3','tj_3_j_3','tj_4_j_3','day_of_week']


### Model validation and evaluation

* LAD model

In [164]:
len(data_training_validation['gap_t(j-1)'])

19421

In [165]:
len(model_lad.predict(feature_numpy_validation))

19421

In [166]:

validation_prediction_lad = (model_lad.predict(feature_numpy_validation) + data_training_validation['gap_t(j-1)'].to_numpy().astype(float)).tolist()
#print validation_prediction[0:100]


In [167]:
validation_target = (target_numpy_validation + data_training_validation['gap_t(j-1)'].to_numpy().astype(float)).tolist()
#print validation_target[0:100]

In [168]:
model_comparison = graphlab.SFrame()
model_comparison['target_value'] = validation_target
model_comparison['model_lad_unmodified'] = validation_prediction_lad

In [169]:
# prediction_modification
validation_prediction_lad_modified = prediction_modification(validation_prediction_lad)
#print validation_prediction_modified[0:100]

In [170]:
(sum, count, mape_lad) = mape(validation_target,validation_prediction_lad_modified)

In [171]:
print "mape_lad = %f" %mape_lad

mape_lad = 0.587063


In [172]:
rmse_lad = np.sqrt(mean_squared_error(validation_target,validation_prediction_lad_modified))

In [173]:
print "rmse_lad = %f" %rmse_lad

rmse_lad = 14.671418


In [174]:
model_comparison['model_lad_modified'] = validation_prediction_lad_modified

* LS model

In [175]:
validation_prediction_ls = model_ls.predict(feature_numpy_validation).tolist()

In [176]:
model_comparison['model_ls_unmodified'] = validation_prediction_ls

In [177]:
validation_prediction_ls_modified = prediction_modification(validation_prediction_ls)

In [178]:
(sum, count, mape_ls) = mape(validation_target,validation_prediction_ls_modified)

In [179]:
print "mape_ls = %f" %mape_ls

mape_ls = 0.522662


In [180]:
rmse_ls = np.sqrt(mean_squared_error(validation_target,validation_prediction_ls_modified))

In [181]:
print "rmse_ls = %f" %rmse_ls

rmse_ls = 42.010155


In [182]:

model_comparison['model_ls_modified'] = validation_prediction_ls_modified

* Graphlab model

In [211]:
validation_prediction_gl = list(model_gl.predict(data_training_validation) + data_training_validation['gap_t(j-1)'])

In [212]:
model_comparison['model_gl_unmodified'] = validation_prediction_gl

In [213]:
validation_prediction_gl_modified = prediction_modification(validation_prediction_gl)

In [214]:
(sum, count, mape_gl) = mape(validation_target,validation_prediction_gl_modified)

In [215]:
print "mape_gl = %f" %mape_gl

mape_gl = 0.573883


In [216]:
rmse_gl = np.sqrt(mean_squared_error(validation_target,validation_prediction_gl_modified))

In [217]:
print "rmse_gl = %f" %rmse_gl

rmse_gl = 11.899280


In [218]:

model_comparison['model_gl_modified'] = validation_prediction_gl_modified

* Export data

In [219]:
model_comparison.export_csv("model_comparison.csv")

In [220]:
#model_comparison

In [221]:
################################################################################

### New boosting: integrate Graphlab rmse model and Scikit lad model
* Low rmse is used to fit big gap numbers
* Low lad is used to fit most of the numbers (here small gap numbers)
* Ask machine to learn how to choose different models

In [222]:
data_boosting = graphlab.SFrame()
data_boosting = data_training_validation['start_district_id', 'time_slot_id', 
                                         'gap_t(j)', 'gap_t(j-1)', 'gap_t(j-2)', 'gap_t(j-3)', 'day_of_week']
data_boosting['gl_ls_predicted'] = validation_prediction_gl_modified
data_boosting['scikit_lad_predicted'] = validation_prediction_lad_modified
features_boosting = ['gl_ls_predicted', 'scikit_lad_predicted']



In [223]:
params = {'n_estimators':100, 'max_depth': 4, 'min_samples_split': 1, 'verbose': True,
          'learning_rate': 0.01, 'loss': 'quantile', 'alpha': 0.5}
model_boosting = ensemble.GradientBoostingRegressor(**params)

feature_numpy_boosting = data_boosting[features_boosting].to_numpy().astype(float)
target_numpy_boosting = data_boosting['gap_t(j)'].to_numpy().astype(float)

model_boosting.fit(feature_numpy_boosting, target_numpy_boosting)

      Iter       Train Loss   Remaining Time 
         1           3.2243            1.09s
         2           3.2157            1.08s
         3           3.2071            1.10s
         4           3.1986            1.10s
         5           3.1901            1.08s
         6           3.1818            1.10s
         7           3.1735            1.05s
         8           3.1577            1.06s
         9           3.1420            1.04s
        10           3.1264            1.04s
        20           2.9778            0.94s
        30           2.8410            0.83s
        40           2.6882            0.66s
        50           2.5617            0.56s
        60           2.4350            0.45s
        70           2.2879            0.32s
        80           2.1646            0.21s
        90           2.0320            0.10s
       100           1.9170            0.00s


GradientBoostingRegressor(alpha=0.5, init=None, learning_rate=0.01,
             loss='quantile', max_depth=4, max_features=None,
             max_leaf_nodes=None, min_samples_leaf=1, min_samples_split=1,
             min_weight_fraction_leaf=0.0, n_estimators=100,
             presort='auto', random_state=None, subsample=1.0,
             verbose=True, warm_start=False)

In [224]:
'''
model_boosting = graphlab.boosted_trees_regression.create(data_boosting, features = features_boosting, target='gap_t(j)', 
                                                          max_iterations = 100,
                                                          max_depth = 9, random_seed = 1)
'''

"\nmodel_boosting = graphlab.boosted_trees_regression.create(data_boosting, features = features_boosting, target='gap_t(j)', \n                                                          max_iterations = 100,\n                                                          max_depth = 9, random_seed = 1)\n"

In [225]:
validation_prediction_boosting = model_boosting.predict(feature_numpy_boosting).tolist()
#validation_prediction_boosting = list(model_boosting.predict(data_training_validation))

In [226]:
validation_prediction_boosting_modified = prediction_modification(validation_prediction_boosting)

In [227]:
(sum, count, mape_boosting) = mape(validation_target,validation_prediction_boosting_modified)

In [228]:
rmse_boosting = np.sqrt(mean_squared_error(validation_target,validation_prediction_boosting_modified))

In [229]:
print "mape_boosting = %f" %mape_boosting
print "rmse_boosting = %f" %rmse_boosting

mape_boosting = 0.424782
rmse_boosting = 32.632881


In [230]:
data_boosting['prediction_boosting'] = validation_prediction_boosting
data_boosting['gap_t(j)','gl_ls_predicted','scikit_lad_predicted','prediction_boosting']

gap_t(j),gl_ls_predicted,scikit_lad_predicted,prediction_boosting
5,8.47780823708,9.09144865451,5.06766903545
2,5.33435690403,5.73950876422,3.31784365065
3,4.15773952007,2.99199503186,2.43592442315
20,7.40858078003,9.81417738877,5.04943948116
1,3.44815015793,2.17741270416,2.17332931596
3,2.21392524242,1.18768249833,1.27164704609
213,229.524843216,214.158436573,104.453338011
2,8.46961545944,4.866583871,4.59229819978
5,5.52556502819,4.03724018328,3.79238464621
6,7.74049800634,6.73946929963,4.99606156597


### Conditional Combination

In [232]:
data_boosting['gap_t(j)','gl_ls_predicted','scikit_lad_predicted','prediction_boosting'].print_rows(num_rows=100)

+----------+-----------------+----------------------+---------------------+
| gap_t(j) | gl_ls_predicted | scikit_lad_predicted | prediction_boosting |
+----------+-----------------+----------------------+---------------------+
|    5     |  8.47780823708  |    9.09144865451     |    5.06766903545    |
|    2     |  5.33435690403  |    5.73950876422     |    3.31784365065    |
|    3     |  4.15773952007  |    2.99199503186     |    2.43592442315    |
|    20    |  7.40858078003  |    9.81417738877     |    5.04943948116    |
|    1     |  3.44815015793  |    2.17741270416     |    2.17332931596    |
|    3     |  2.21392524242  |    1.18768249833     |    1.27164704609    |
|   213    |  229.524843216  |    214.158436573     |    104.453338011    |
|    2     |  8.46961545944  |     4.866583871      |    4.59229819978    |
|    5     |  5.52556502819  |    4.03724018328     |    3.79238464621    |
|    6     |  7.74049800634  |    6.73946929963     |    4.99606156597    |
|    7     |

In [231]:
# search for threhold
for i in range(1, int(max(validation_prediction_lad_modified)) + 1):
    prediction_conditional_combination = []
    for j in range(len(validation_prediction_lad_modified)):
        if validation_prediction_lad_modified[j] <= i:
            prediction_conditional_combination.append(validation_prediction_lad_modified[j])
        else:
            prediction_conditional_combination.append(validation_prediction_gl_modified[j])
    (sum, count, mape_conditional_combination) = mape(validation_target, prediction_conditional_combination)
    print mape_conditional_combination

0.570259125157
0.567298727425
0.565697333198
0.566166674844
0.568487390033
0.568749753305
0.570549214444
0.569713403601
0.570640275905
0.570489051228
0.570413912739
0.568531330152
0.567569017628
0.568041407927
0.567866493822
0.565561355237
0.565018214249
0.566323210447
0.566608338985
0.56814673299
0.567927496561
0.568074661623
0.56829985447
0.568432038244
0.568420245469
0.570034839604
0.570186912503
0.571003556117
0.571384006716
0.571656534604
0.571866086603
0.57240090654
0.573570847033
0.573611927866
0.574113183452
0.574535545975
0.574790989261
0.57485442174
0.574949780653
0.575046138813
0.575157133471
0.57540221111
0.57539431799
0.575629223064
0.575607530628
0.57571374491
0.575718388395
0.575742923591
0.575709311349
0.575672925493
0.575787469576
0.575739472862
0.575738551285
0.575945192897
0.57597491309
0.575956897745
0.575999685151
0.575985846425
0.576097158998
0.576112264835
0.576076462607
0.576561991985
0.5763307541
0.576417534778
0.57648419792
0.577021170855
0.576764365974
0.5767

ValueError: I/O operation on closed file

In [ ]:
# search for threhold
mape_smallest = 1
i_corresponding = 1
for i in range(1, int(max(validation_prediction_gl_modified)) + 1):
    prediction_conditional_combination = []
    for j in range(len(validation_prediction_gl_modified)):
        if validation_prediction_gl_modified[j] >= i:
            prediction_conditional_combination.append(validation_prediction_gl_modified[j])
        else:
            prediction_conditional_combination.append(validation_prediction_lad_modified[j])
    (sum, count, mape_conditional_combination) = mape(validation_target, prediction_conditional_combination)
    if mape_conditional_combination < mape_smallest:
        mape_smallest = mape_conditional_combination
        i_corresponding = i
print mape_smallest
print i_corresponding

prediction_conditional_combination = []
for j in range(len(validation_prediction_gl_modified)):
    if validation_prediction_gl_modified[j] >= i_corresponding:
        prediction_conditional_combination.append(validation_prediction_gl_modified[j])
    else:
        prediction_conditional_combination.append(validation_prediction_lad_modified[j])
    
data_boosting['prediction_conditional_combination'] = prediction_conditional_combination

In [ ]:
data_boosting['gap_t(j)','gl_ls_predicted','scikit_lad_predicted','prediction_boosting', 'prediction_conditional_combination']\
.export_csv("model_comparison_2.csv")

### Model prediction

In [ ]:
prediction_items = []
fhand = open("read_me_1.txt")
for line in fhand:
    line_splitted = line.strip().split('-')
    prediction_items.append(((line_splitted[0] + '-' + line_splitted[1] + '-' + line_splitted[2]), line_splitted[3]))
fhand.close()
#prediction_items

In [ ]:
data_test_set_1_filtered = graphlab.SFrame()
for (date, time_slot_id) in prediction_items:
    data_test_set_1_filtered = data_test_set_1_filtered.append(
    data_test_set_1[(data_test_set_1['date'] == date) & (data_test_set_1['time_slot_id'] == int(time_slot_id))])

In [ ]:
feature_numpy_test = data_test_set_1_filtered[features].to_numpy()

In [ ]:
test_prediction = model.predict(feature_numpy_test).tolist()

In [ ]:
test_prediction_modified = prediction_modification(test_prediction)

# test_prediction_modified[0:100]

In [ ]:
data_test_set_1_filtered['prediction'] = test_prediction_modified

In [ ]:
#data_test_set_1_filtered

### Make submission file

In [ ]:
def make_submission(result, filename='submission.txt'):
    output_file = open(filename,'w')
    for row in data_test_set_1_filtered:
        output_file.write(str(row['start_district_id']) + ',' + row['date'] + '-' \
                          + str(row['time_slot_id']) + ',' + str(row['prediction']) + '\n')
    output_file.close()

In [ ]:
make_submission(data_test_set_1_filtered)